In [31]:
import chromadb
from chromadb.api.models.Collection import Collection
from llama_cpp import Llama
from qdrant_client import QdrantClient
from typing import List, Optional, Union
from langchain.vectorstores import Qdrant, Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)

In [2]:
collection = "all-documents"
model = "models/saiga2_7b_gguf/model-q2_K.gguf"
EMBEDDER_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
llama_model = Llama(
    model_path=model,
    n_ctx=2000,
    n_parts=1,
)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDER_NAME)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/saiga2_7b_gguf/model-q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q2_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q3_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q3_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q3_K     [ 14336,  4096,     1,     1 

In [13]:
LOADER_MAPPING: dict = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

def load_single_document(file_path: str):
    """
    Загружаем один документ.
    :param file_path:
    :return:
    """
    ext: str = "." + file_path.rsplit(".", 1)[-1]
    assert ext in LOADER_MAPPING
    loader_class, loader_args = LOADER_MAPPING[ext]
    loader = loader_class(file_path, **loader_args)
    return loader.load()[0]

In [78]:
query = "Какие требования к актуализации ВНД"
path = "load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx"
documents: List[Document] = [load_single_document(path)]

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1117, which is longer than the specified 500
Created a chunk of size 1975, which is longer than the specified 500
Created a chunk of size 1494, which is longer than the specified 500
Created a chunk of size 2952, which is longer than the specified 500
Created a chunk of size 1492, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 1099, which is longer than the specified 500
Created a chunk of size 2596, which is longer than the specified 500


In [ ]:
# LANGCHAIN QDRANT

In [87]:
client = QdrantClient(path="./qdrant_langchain_test")
qdrant = Qdrant(client, collection_name=collection, embeddings=embeddings)

RuntimeError: Storage folder ./qdrant_langchain_test is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [88]:
qdrant = qdrant.from_documents(
    docs,
    embeddings,
    path="./qdrant_langchain_test",
    collection_name=collection,
)
qdrant

In [91]:
found_docs = qdrant.similarity_search(query)

print(found_docs[0].metadata)
print(found_docs[0].page_content)

{'source': 'load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx'}
Необходимость в разработке / актуализации ВНД по приведенным выше основаниям выявляется владельцами и участниками процессов, органами управления Общества, уполномоченными государственными и иными организациями / лицами. 

Выявленная потребность в регламентации процессов может при необходимости фиксироваться в плане-графике разработки / актуализации ВНД по направлению деятельности. 

Требования к разработке проекта ВНД

Процесс разработки ВНД в Обществе может быть инициирован:


In [ ]:
# LANGCHAIN CHROMA

In [58]:
persistent_client = chromadb.PersistentClient(path="./chroma_langchain_test")
db = Chroma(
    client=persistent_client,
    collection_name=collection,
    embedding_function=embeddings,
)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [59]:
db = Chroma(persist_directory="./chroma_langchain_test", embedding_function=embeddings)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [60]:
db = Chroma.from_documents(
    docs, 
    embeddings,
    persist_directory="./chroma_langchain_test",
    collection_name=collection,
)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [ ]:
docs = db.similarity_search(query)

# print results
print(docs[0].metadata)
print(docs[0].page_content)

In [ ]:
# CHROMA

In [72]:
client: chromadb.PersistentClient = chromadb.PersistentClient(path="./chroma_test")
db: Collection = client.get_or_create_collection(collection)
db

Collection(name=all-documents)

In [69]:
ids: List[str] = [
    f"{doc.metadata['source'].split('/')[-1].replace('.txt', '')}{i}"
    for i, doc in enumerate(docs)
]

db.add(
    documents=[doc.page_content for doc in docs],
    metadatas=[doc.metadata for doc in docs],
    ids=ids
)

In [73]:
docs = db.query(
    query_texts=[query],
    n_results=4
)

In [74]:
data: dict = {}
for doc, text in zip(docs["metadatas"][0], docs["documents"][0]):
    document: str = f'Документ - {doc["source"].split("/")[-1]}'
    if document in data:
        data[document] += "\n" + text
    else:
        data[document] = text
list_data: list = [f"{doc}\n\n{text}" for doc, text in data.items()]
print("\n\n\n".join(list_data))

Документ - ПН_РСК_001_02_Положение_о_системе_ВНД.docx

- документы 4-го уровня, определяющие последовательность выполнения действий внутри одного функционального блока/подразделения.

В Обществе предусмотрены следующие виды ВНД:

Таблица №1
ВНД утверждается и вводится в действие органом управления / должностным лицом в соответствии со следующими правилами:
Для удобства управления все ВНД, создаваемые в Обществе, классифицируются по видам, типам доступа и сроку действия.

В Обществе, в зависимости от назначения, разрабатываются ВНД следующих видов:
По результатам разработки все проекты ВНД должны пройти процедуру предварительного согласования, включающую рассмотрение:

непосредственным руководителем Разработчика на соответствие содержания ВНД целям его разработки;
